In [ ]:
# import sys
# import gymnasium as gym
# sys.modules["gym"] = gym
import gym
gym.__version__

In [ ]:
from stable_baselines3 import PPO
from stable_baselines3.common.monitor import Monitor
from stable_baselines3.common.vec_env import DummyVecEnv, SubprocVecEnv
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.callbacks import EvalCallback

In [ ]:
import numpy as np
import torch as th
from torch import nn
import matplotlib.pyplot as plt
from pprint import pprint

In [ ]:
import gnwrapper

In [ ]:
env_id = "CarRacing-v0"
NUM_CPU = 16  # Number0of processes to use

In [ ]:
from CarRacing_utils import CarRacingGroundTruthXYObsWrapper
def wrapper(env):
    env = CarRacingGroundTruthXYObsWrapper(env) 
    env = gnwrapper.Animation(env)
    return env

def wrapper_image(env):
    env = gym.wrappers.gray_scale_observation.GrayScaleObservation(env, keep_dim=False)
    env = gym.wrappers.FrameStack(env, 2)
    env = gnwrapper.Animation(env)
    return env

In [ ]:
# expert = PPO.load("./policy/ppo_CarRacing_expert-1kk.zip", print_system_info=True)
expert = PPO.load("./policy/ppo_CarRacing_XY_expert33046.zip", print_system_info=True)

In [ ]:
from imitation.data import rollout
from imitation.data.wrappers import RolloutInfoWrapper
import dataclasses
from stable_baselines3.common.vec_env import VecTransposeImage, VecFrameStack

class TransformaObservacions(gym.ObservationWrapper):
    def __init__(self, env):
        super().__init__(env)
        
        image_width = self.observation_space.shape[0]
        image_height = self.observation_space.shape[1]
        n_chanel = self.observation_space.shape[2]
        
        new_obs_shape = (n_chanel, image_width, image_height)
        
        self.observation_space = gym.spaces.Box(shape=new_obs_shape, low=0, high=255)

    def observation(self, observ):
        return np.transpose(observ, (2, 0, 1))  
    
def make_env(env_id):
    def _init():
        env = gym.make(env_id)
        env = gym.wrappers.gray_scale_observation.GrayScaleObservation(env, keep_dim=False)
        env = gym.wrappers.FrameStack(env, 2)
        env = RolloutInfoWrapper(env) # Wrapper to save origin obs
        env = wrapper(env) # Wrapper Obs
        return env
    
    return _init

# env = gym.wrappers.gray_scale_observation.GrayScaleObservation(env, keep_dim=True)
# env = RolloutInfoWrapper(env) # Wrapper to save origin obs
# env = wrapper(env) # Wrapper Obs
# env = DummyVecEnv([lambda: env]) # Vectorized env

bc_env = DummyVecEnv([make_env(env_id)]*NUM_CPU)
bc_env.observation_space

In [ ]:
NUM_EPISODES = 1000
rng = np.random.default_rng()
rollouts = rollout.rollout(
    expert,
    bc_env,
    rollout.make_sample_until(min_timesteps=None, min_episodes=NUM_EPISODES),
    rng = rng,
    unwrap = True,
)

In [ ]:
top_rollouts = [rollout for rollout in rollouts if len(rollout.rews) < 880]
transitions = rollout.flatten_trajectories(top_rollouts)

In [ ]:
pprint(rollout.rollout_stats(rollouts))
pprint(rollout.rollout_stats(top_rollouts))

In [ ]:
print(
    f"""The `rollout` function generated a list of {len(rollouts)} {type(rollouts[0])}.
After flattening, this list is turned into a {type(transitions)} object containing {len(transitions)} transitions.
The transitions object contains arrays for: {', '.join(transitions.__dict__.keys())}."
"""
)

In [ ]:
from imitation.algorithms import bc
from stable_baselines3.common.policies import ActorCriticCnnPolicy

# env = gym.make(env_id)
# env = gym.wrappers.gray_scale_observation.GrayScaleObservation(env, keep_dim=False)
# env = TransformaObservacions(env)
# env = gym.wrappers.FrameStack(env, 2)
# env = gnwrapper.Animation(env)
# env = DummyVecEnv([lambda: env])
# env = VecTransposeImage(env)

env = make_vec_env(env_id, wrapper_class=wrapper_image, n_envs=1)

bc_trainer = bc.BC(
    observation_space=env.observation_space,
    action_space=env.action_space,
    demonstrations=transitions,
    rng=rng,
    batch_size=4096,
    policy=ActorCriticCnnPolicy(
                         observation_space=env.observation_space,
                         action_space=env.action_space,
                         lr_schedule=lambda _: th.finfo(th.float32).max,
                         net_arch=dict(pi=[256], vf=[256]),
                         # net_arch=[256, 256],
                         activation_fn = nn.LeakyReLU, # nn.GELU, # nn.LeakyReLU,
                         ortho_init=False,
                         )
)

In [ ]:
bc_trainer.train(n_epochs=100)

In [ ]:
env = gym.make(env_id)
env = wrapper_image(env)
env = gym.wrappers.RecordVideo(env, 'video')

obs = env.reset()
while True:
    action, _states = bc_trainer.policy.predict(np.asarray(obs))
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        break
    
env.close()

In [ ]:
env = gym.make(env_id)
env = wrapper(env)
env = gym.wrappers.RecordVideo(env, 'video')

obs = env.reset()
while True:
    action, _states = expert.predict(obs.copy())
    obs, rewards, dones, info = env.step(action)
    env.render()
    if dones:
        break
    
env.close()

In [ ]:
from stable_baselines3.common.evaluation import evaluate_policy

In [ ]:
from stable_baselines3.common.vec_env import VecFrameStack
def wrapper_image2(env):
    env = gym.wrappers.gray_scale_observation.GrayScaleObservation(env, keep_dim=True)
    env = gnwrapper.Animation(env)
    return env

test_env_gt = make_vec_env(env_id, wrapper_class=wrapper, n_envs=10)
test_env = make_vec_env(env_id, wrapper_class=wrapper_image2, n_envs=10)
test_env = VecFrameStack(test_env, 2)

expert_reward, expert_reward_std = evaluate_policy(expert, test_env_gt, 10)
bc_reward, bc_reward_std = evaluate_policy(bc_trainer.policy, test_env, 10)
# noob_reward, noob_reward_std = evaluate_policy(noob, test_env, 10)

In [ ]:
print(f'expert reward {expert_reward:.2f} +/- {expert_reward_std:.2f}')
print(f'BC reward {bc_reward:.2f} +/- {bc_reward_std:.2f}')
# print(f'noob reward {noob_reward} +/- {noob_reward_std}')

In [ ]:
save(opt.log_prefix + opt.model_name)

In [ ]:
bc_trainer.save_policy('policy/bc1')

In [ ]:
th.load('policy/bc1.zip').predict(obs.copy())